# Segmenting and Clustering Neighborhoods in Toronto, CA
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## Step 1: Scrape Wikipedia for Neighborhood Data using BeautifulSoup

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

To create the above dataframe:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

**Note:** There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe. Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

In [2]:
# Install BeautifulSoup and reequests for web scraping
!pip install BeautifulSoup4
!pip install requests

### Part A: Retrieve the Data from the Web Page

1. Get HTML from Wikipedia
2. Use BeautifulSoup to parse the retrieved data
3. Convert and store the data into a pandas dataframe

In [5]:
# Preliminary work
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Retrieve HTML source from Wikipedia site and create the Soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

# Using the Soup object, iterate through the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    # First row of data is the column header
    if (index == 0):
        columns = section
    else:
        data.append(section)

# Finally, convert the list into a Pandas dataframe
df = pd.DataFrame(data = data,columns = columns)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Part B: Clean Up the Data

1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
2. More than one neighborhood can exist in one postal code area. **NOTE:** This is already done, where the different neighborhoods are separated by a '/'.
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [11]:
# 1. Remove Boroughs that are 'Not assigned' or NULL
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [21]:
# Remove duplicates
df = df.drop_duplicates()

# Update index to be postcode if it isn't already
if(df.index.name != 'Postal code'):
    df = df.set_index('Postal code')
    
df.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [23]:
# If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough
df['Neighborhood'].replace("Not assigned", df["Borough"], inplace=True)
df.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [24]:
df.shape

(98, 2)